<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/Sntence_entailment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**work in progress**

# Data Exploration

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# file upload while using Google Colab
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
!unzip train.zip

In [ ]:
train = pd.read_csv("train.csv")

In [ ]:
train_filtered  = train.filter(['premise','hypothesis','label'])

In [ ]:
train_filtered.head()

,premise,hypothesis,label
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2
2,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,0
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0
4,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,1


In [ ]:
#get cell value
train.iloc[1]['premise']
train.iloc[1]['hypothesis']

In [ ]:
dataset = nlp.Dataset.from_pandas(train_filtered)
ds_splitted = dataset.train_test_split(test_size=0.1)

In [ ]:
ds_splitted['train']['premise'][1]

"because i always had to do it and so i just pay someone else to do it and they do the they do the cutting they fertilize they um edge and um i think this year i'm going to have some landscaping put in"

# Model

In [ ]:
%%capture
!pip install pytorch_lightning
!pip install transformers
!pip install nlp

In [ ]:
import torch as th
import pytorch_lightning as pl
import nlp
import transformers

In [ ]:
#example of tokenizing 2 sentences into
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
sequence_a = ds_splitted['train']['premise'][1]
sequence_b = ds_splitted['train']['hypothesis'][1]
encoded_dict = tokenizer(sequence_a, sequence_b,                     
                         max_length=128, 
                         truncation = True,
                         pad_to_max_length=True,
                         return_tensors = 'pt')
#decoded = tokenizer.decode(encoded_dict["input_ids"])
print(type(encoded_dict))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [ ]:
model = transformers.BertModel.from_pretrained('bert-base-uncased')
outputs = model(**encoded_dict)

In [ ]:
outputs[1].shape

torch.Size([1, 768])

In [ ]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs[0]

In [ ]:
type(inputs)

transformers.tokenization_utils_base.BatchEncoding

In [ ]:
class Sentences(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.brt = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.layer1 = th.nn.Linear(64,32)
        self.layer2 = th.nn.Linear(32,3)
        self.loss = th.nn.CrossEntropyLoss(reduction='none')

    def prepare_data(self):
        tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

        def _tokenize(x):
            print('in tokenize')
            return tokenizer(
                    x['premise'],
                    x['hypothesis'],
                    max_length=64, 
                    truncation = True,
                    pad_to_max_length=True)
            
        def _prepare_ds():
            df = pd.read_csv('train.csv', nrows=50)
            df_filtered  = df.filter(['premise','hypothesis','label'])
            dataset = nlp.Dataset.from_pandas(df_filtered)
            ds_flt = dataset.train_test_split(test_size=0.1)
            print(ds_flt)
            print(ds_flt['train']['premise'][2])
            ds_flt['train'] = ds_flt['train'].map(_tokenize, batched=True)
            print('training portion of the dataset'.format(ds_flt['train'][0]))
            ds_flt['train'].set_format(type='torch',columns = ['input_ids','token_type_ids','label','attention_mask'])
            print(ds_flt['train'][0])
            ds_flt['test'] = ds_flt['test'].map(_tokenize, batched=True)
            ds_flt['test'].set_format(type='torch',columns = ['input_ids','token_type_ids','label','attention_mask'])
            print(ds_flt['test'][0])
            #print('ds_train type - {}'.format(type(ds_flt['train'])))
            #print('ds_test type - {}'.format(type(ds_flt['test'])))

            return ds_flt['train'], ds_flt['test']

        self.train_ds, self.test_ds = _prepare_ds()      

    def forward(self, input_ids, masks, token_type_ids):

        logits = self.brt(input_ids, masks, token_type_ids)
        print(type(logits))
        #out = self.layer1(logits)
        #out = th.nn.reLU(out)
        #out = self.layer2(out)
        #print ('logits- {} -- other stuff'.format(logits))
        return logits

    def training_step(self, batch, batch_idx):
        print('training step function')
        logits = self.forward(batch['input_ids'],batch['attention_mask'], batch['token_type_ids'])
        loss = self.loss(logits, batch['label']).mean()

        return {'loss': loss, 'log': {'train_loss': loss}}

    def validation_step(self, batch, batch_idx):
        #print('validation step function')
        logits = self.forward(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
        loss = self.loss(logits, batch['label'])
        acc = (logits.argmax(-1) == batch['label']).float()

        return {'loss': loss, 'acc': acc}

    def validation_epoch_end(self, outputs):
        loss = th.cat([o['loss'] for o in outputs], 0).mean()
        acc = th.cat([o['acc'] for o in outputs], 0).mean()
        out = {'val_loss': loss, 'val_acc': acc}
        return {**out, 'log': out}

    def train_dataloader(self):
        #print('train_dataloader')
        return th.utils.data.DataLoader(
                self.train_ds,
                batch_size=1,
                drop_last=True,
                shuffle=True,
                )

    def val_dataloader(self):
        #print('val_dataloader')
        return th.utils.data.DataLoader(
                self.test_ds,
                batch_size=32,
                drop_last=False,
                shuffle=True,
                )

    def configure_optimizers(self):
        return th.optim.SGD(
            self.parameters(),
            lr=0.01,
            momentum=0.9,
        )

In [ ]:

def model_fit():
    model = Sentences()
    trainer = pl.Trainer(
        #default_root_dir='logs',
        gpus=(1 if th.cuda.is_available() else 0),
        max_epochs=4,
        fast_dev_run=False,
        logger=pl.loggers.TensorBoardLogger('logs/', name='imdb'),
    )
    trainer.fit(model)

!rm -rf ./logs/

model_fit()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores



{'train': Dataset(features: {'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 45), 'test': Dataset(features: {'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 5)}
Lạnh hơn và xa hơn bao giờ hết đã phát triển tiếng nói của lãnh chúa.
in tokenize


in tokenize

training portion of the dataset
{'label': tensor(2), 'input_ids': tensor([ 101, 1045, 2572, 4527, 2295, 2008, 2057, 2079, 2031, 2061, 2116, 2008,
        2024, 1999, 4331, 2091, 2182,  102, 1045, 2572, 4527, 2008, 2025, 2116,
        1997, 2068, 2024, 1999, 4331, 2091, 2182, 1012,  102,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}
in tokenize


in tokenize



/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name   | Type             | Params
--------------------------------------------
0 | brt    | BertModel        | 109 M 
1 | layer1 | Linear           | 2 K   
2 | layer2 | Linear           | 99    
3 | loss   | CrossEntropyLoss | 0     


{'label': tensor(0), 'input_ids': tensor([ 101, 2017, 2113, 2027, 2064, 1005, 1056, 2428, 6985, 3209, 2066, 8307,
        4961, 7910, 2360, 2026, 2287, 2017, 2113, 3398,  102, 2027, 2064, 1005,
        1056, 6985, 3209, 2138, 1997, 2037, 2287, 1012,  102,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


<class 'tuple'>


AttributeError: ignored

# Sandbox